In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.1 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=96faa56e4218f4192c498856fd8e2cbec9bdc570bc79d320b02e30a094b7c5f3
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten,BatchNormalization,Dropout,Activation ,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import CategoricalCrossentropy
import PIL
from keras.callbacks import EarlyStopping
from keras import optimizers


import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Conv2D, Flatten, BatchNormalization, Dropout, Activation, MaxPooling2D, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import CategoricalCrossentropy
import PIL
import wandb
import os
from wandb.keras import WandbCallback
from keras.callbacks import EarlyStopping
from keras import optimizers

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.optimizers import SGD, Adam, RMSprop

import tensorflow as tf

In [ ]:
class Wandb_tuning:
    def __init__(self):
        wandb.login()
    
    def train(self):
        default_hyperparams = dict(
            tr_model = 'VGG16',
            learning_rate=0.01,
            optimizer = 'Adam',
            # freeze = 0.5,
            epochs = 5,
            data_aug = 0,
            batch_size = 32,
            # dp=0.2
        )    

        wandb.init(config = default_hyperparams)
        config = wandb.config
        wandb.run.name = str(config.tr_model)+'_'+'Learning_rate='+str(config.learning_rate)+'_'+'optimizer='+str(config.optimizer)+'_epochs='+str(config.epochs)+'_batch_size='+str(config.batch_size)
        wandb.run.save()

        train_data_dir= '/content/drive/MyDrive/LandT/train/'
        val_data_dir= '/content/drive/MyDrive/LandT/valid/'
        # batch_size = config.batch_size
        if config.data_aug:      
            train_datagen = ImageDataGenerator(
                rescale=1./255,
                rotation_range=45, 
                horizontal_flip=True, 
                vertical_flip=True,
                validation_split=0.1)
        else:      
            train_datagen = ImageDataGenerator(
                          rescale=1./255)
                        # validation_split=0.1) # set validation split
        train_it = train_datagen.flow_from_directory(
            train_data_dir,
            target_size=(224,224),
            batch_size=config.batch_size,
            class_mode='binary',
            subset='training') # set as training data

        val_it = train_datagen.flow_from_directory(
            val_data_dir, # same directory as training data
            target_size=(224,224),
            batch_size=config.batch_size,
            class_mode='binary',
            subset='validation') # set as validation data
        
        trained_model = VGG16(input_shape = [224, 224] + [3], weights='imagenet', include_top=False)
        for layer in trained_model.layers[:129]:
            layer.trainable = False
        for layer in trained_model.layers[129:]:
            layer.trainable = True
          
        x = trained_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(1024, activation='relu')(x)
        predictions = Dense(1, activation='sigmoid')(x)
        model = Model(inputs = trained_model.input, outputs = predictions)

        if config.optimizer == 'Adam':
            if config.learning_rate == 0:
                model.compile(
                    optimizer='adam',
                    loss='binary_crossentropy', metrics=['acc',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
                ) 
            else: 
                model.compile(optimizer=Adam(lr = config.learning_rate, decay = 1e-6), loss='binary_crossentropy', metrics=['acc',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        elif config.optimizer == 'rmsprop':
            if config.learning_rate == 0:
                model.compile(
                    optimizer='rmsprop',
                    loss='binary_crossentropy', metrics=['acc',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
                )  
            else:
                model.compile(optimizer=RMSprop(lr = config.learning_rate, decay = 1e-6), loss='binary_crossentropy', metrics=['acc',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        elif config.optimizer == 'sgd':
            if config.learning_rate == 0:
                model.compile(
                    optimizer='sgd',
                    loss='binary_crossentropy', metrics=['acc',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
                )  
            else:
                model.compile(optimizer=SGD(lr = config.learning_rate, momentum=0.9), loss='binary_crossentropy', metrics=['acc',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        
        model.fit(
              train_it,
              steps_per_epoch = train_it.samples // config.batch_size,
              validation_data = val_it, 
              validation_steps = val_it.samples // config.batch_size,
              epochs = config.epochs,
              verbose=1,
              callbacks=[WandbCallback(data_type='image',validation_data = val_it, verbose=1),EarlyStopping(patience=10,restore_best_weights=True)])

    def get_sweep_config(self):
        sweep_config = {
            'method': 'bayes',         
            'metric': {
                'name': 'val_accuracy',     
                'goal': 'maximize'      
            },
            'parameters': {
                'learning_rate': {
                    'values': [0, 1e-3, 5*1e-4, 1e-4]   # 0 implies default learning rate set by model
                },
                'epochs': {
                    'min': 5,
                    'max': 20
                },
                'data_aug': {
                    'values': [0,1]
                },
                'optimizer' : {
                    'values': ['Adam', 'rmsprop', 'SGD']
                },
                'tr_model' : {
                    'values': ['VGG16'] #['InceptionV3', 'Xception', 'InceptionResNetV2']
                },
                'batch_size': {
                    'values': [32, 64]
                }
            }
        }
        return sweep_config
    
    def start_training(self, project_name, runs):
        sweep_id = wandb.sweep(self.get_sweep_config, project=project_name)
        wandb.agent(sweep_id, function=self.train, count = runs)


In [ ]:
obj = Wandb_tuning()

In [ ]:
obj.start_training("LnT_Xception_Tuning", 20)

Create sweep with ID: 2rqq8jb4
Sweep URL: https://wandb.ai/sunil_sk/LnT_Xception_Tuning/sweeps/2rqq8jb4


wandb: Agent Starting Run: k1qxqgks with config:
wandb: 	batch_size: 64
wandb: 	data_aug: 1
wandb: 	epochs: 18
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: rmsprop
wandb: 	tr_model: VGG16
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 540 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/18
1/1 [==============================] - 2s 2s/step


8/8 [==============================] - 17s 1s/step - loss: 0.6671 - acc: 0.6071 - precision: 0.6415 - recall: 0.4397
Epoch 2/18
1/1 [==============================] - 0s 30ms/step


8/8 [==============================] - 8s 938ms/step - loss: 0.6219 - acc: 0.7752 - precision: 0.7811 - recall: 0.7647
Epoch 3/18
1/1 [==============================] - 0s 36ms/step


...

wandb: WARNING No validation_data set, pass a generator to the callback.


9/9 [==============================] - 3s 293ms/step - loss: 0.1409 - acc: 0.9552 - precision: 0.9630 - recall: 0.9489
Epoch 12/12
9/9 [==============================] - ETA: 0s - loss: 0.1484 - acc: 0.9496 - precision: 0.9762 - recall: 0.9213

wandb: WARNING No validation_data set, pass a generator to the callback.


9/9 [==============================] - 3s 290ms/step - loss: 0.1484 - acc: 0.9496 - precision: 0.9762 - recall: 0.9213


acc,▁▆▇▇▇▇██████
epoch,▁▂▂▃▄▄▅▅▆▇▇█
loss,█▅▄▃▂▂▂▂▁▁▁▁
precision,▁▆▇▇▇▇██████
recall,▁▅▄▇▇▆▆██▇█▇
acc,0.94963
epoch,11
loss,0.14837
precision,0.97619
recall,0.92135


In [ ]:

train_data_dir= '/content/drive/MyDrive/LandT/train/'
val_data_dir= '/content/drive/MyDrive/LandT/valid/'

batch_size=64

if 0:
    
  train_datagen = ImageDataGenerator(
  rescale=1./255,
  rotation_range=45, 
  #width_shift_range=.15, 
  #height_shift_range=.15, 
  horizontal_flip=True, 
  #zoom_range=0.5,
  vertical_flip=True,
  validation_split=0.1) 

else:
    
  train_datagen = ImageDataGenerator(
  rescale=1./255)
  #validation_split=0.1) # set validation split

  

train_it = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224,224),
    batch_size=batch_size,
    class_mode='binary',
    subset='training') # set as training data

val_it = train_datagen.flow_from_directory(
    val_data_dir, # same directory as training data
    target_size=(224,224),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation') # set as validation data

# Your model here ...
trained_model = Xception(input_shape = [224, 224] + [3], weights='imagenet', include_top=False)
  
for layer in trained_model.layers[:129]:
  layer.trainable = False

for layer in trained_model.layers[129:]:
  layer.trainable = True
  
x = trained_model.output
x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs = trained_model.input, outputs = predictions)
model.compile(optimizer=Adam(learning_rate = 0.001, decay = 1e-6), loss='binary_crossentropy', metrics=['acc',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

Found 600 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
83683744/83683744 [==============================] - 1s 0us/step


In [ ]:
#model.compile(tf.optimizers.Adam(learning_rate=0.0001, decay=1e-6), loss=CategoricalCrossentropy(), metrics='acc')
model.fit(
train_it,
steps_per_epoch = train_it.samples //batch_size,
validation_data = val_it, 
validation_steps = val_it.samples // batch_size,
epochs = 25,
verbose=1,
callbacks=[EarlyStopping(patience=10,restore_best_weights=True)])

Epoch 1/25
9/9 [==============================] - ETA: 0s - loss: 7.1158e-04 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000 

9/9 [==============================] - 121s 13s/step - loss: 7.1158e-04 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 2/25
9/9 [==============================] - ETA: 0s - loss: 3.4463e-04 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000 

9/9 [==============================] - 120s 13s/step - loss: 3.4463e-04 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 3/25
9/9 [==============================] - ETA: 0s - loss: 2.5434e-04 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000 

9/9 [==============================] - 121s 13s/step - loss: 2.5434e-04 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 4/25
9/9 [==============================] - ETA: 0s - loss: 2.0305e-04 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000 

9/9 [==============================] - 132s 15s/step - loss: 2.0305e-04 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 5/25
9/9 [==============================] - ETA: 0s - loss: 3.2458e-04 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000 

...

9/9 [==============================] - 119s 13s/step - loss: 3.8853e-05 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 22/25
9/9 [==============================] - ETA: 0s - loss: 3.3303e-04 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000 

9/9 [==============================] - 116s 14s/step - loss: 3.3303e-04 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 23/25
9/9 [==============================] - ETA: 0s - loss: 9.0663e-05 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000 

9/9 [==============================] - 118s 13s/step - loss: 9.0663e-05 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 24/25
9/9 [==============================] - ETA: 0s - loss: 3.7445e-05 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000 

9/9 [==============================] - 118s 13s/step - loss: 3.7445e-05 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 25/25
9/9 [==============================] - ETA: 0s - loss: 3.0431e-05 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000 

9/9 [==============================] - 116s 13s/step - loss: 3.0431e-05 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255.)
test_data_dir = '/content/drive/MyDrive/LandT/test/'

test_it = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224,224),
    batch_size=1,
    class_mode='binary',
    shuffle=False
)

test_loss, test_accuracy,test_pre,test_recall  = model.evaluate(test_it)
print('test_loss = ', test_loss, ', test_accuracy = ',  test_accuracy , 'test_precision=', test_pre , 'test_recall=', test_recall)

Found 200 images belonging to 2 classes.
200/200 [==============================] - 50s 242ms/step - loss: 0.0091 - acc: 0.9950 - precision: 0.9901 - recall: 1.0000
test_loss =  0.009108847938477993 , test_accuracy =  0.9950000047683716 test_precision= 0.9900990128517151 test_recall= 1.0


In [ ]:
import pickle
filename = 'l&T_Xception.sav'
pickle.dump(model, open(filename, 'wb'))
# #Load model.... 


In [ ]:
import pickle
model = pickle.load(open('/content/drive/MyDrive/classification_model.sav', 'rb'))